In [2]:
import cv2 as cv
import numpy as np

In [36]:
class Detector:
    @staticmethod
    def nothing(x):
        pass
    @staticmethod
    def drawRectangles(frame, contours,area_l_thresh,area_u_thresh):
        x_list=[]
        y_list=[]
        for contour in contours:
            (x, y, w, h) = cv.boundingRect(contour)
            if cv.contourArea(contour) > area_l_thresh and cv.contourArea(contour) < area_u_thresh:
                cv.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)
                cv.putText(frame,"Status: {}".format("Detected"), (10,30), cv.FONT_HERSHEY_SIMPLEX,
                                        1, (0, 0, 255), 3)
                x_list.append((x, x+w))
                y_list.append((y, y+h))
        return x_list, y_list
    @staticmethod
    def drawLineRect(frame, x_list, y_list, line_colour = (0,0,255) , line_thickness = 1, font = cv.FONT_HERSHEY_SIMPLEX):
        frame_lined = frame.copy()
        center_x = int(frame_lined.shape[1]/2)
        center_y = int(frame_lined.shape[0]/2)
        center_coordinate = (center_x, center_y)
        for xs, ys in zip(x_list,y_list):
            xc = int((xs[0] + xs[1])/2)
            yc = int((ys[0] + ys[1])/2)
            obj_center_coordinate = (xc, yc)
            line_lenght = ((xc -center_x)**2 + (yc -center_y)**2)**(1/2)
            cv.line(frame_lined, center_coordinate, obj_center_coordinate, line_colour, line_thickness)
            cv.putText(frame_lined,"Len: {:.2f}".format(line_lenght), (xc,yc), font,
                                            1, (0, 0, 255), 2)
        return frame_lined
    @staticmethod
    def drawLine(frame, to_loc, line_colour = (0,0,255) , line_thickness = 1, font = cv.FONT_HERSHEY_SIMPLEX):
        frame_lined = frame.copy()
        center_x = int(frame_lined.shape[1]/2)
        center_y = int(frame_lined.shape[0]/2)
        center_coordinate = (center_x, center_y)
        obj_center_coordinate = (to_loc[0], to_loc[1])
        line_lenght = ((to_loc[0] -center_x)**2 + (to_loc[1] -center_y)**2)**(1/2)
        cv.line(frame_lined, center_coordinate, obj_center_coordinate, line_colour, line_thickness)
        cv.putText(frame_lined,"Len: {:.2f}".format(line_lenght), (to_loc[0],to_loc[1]), font,
                                            1, (0, 0, 255), 2)
        return frame_lined
    @staticmethod
    def findContours(frame,l_bound,u_bound,
                        dilate=True,
                        dilate_iteration = 1,
                        draw_contours=False,
                        draw_rectangle=True,
                        filter_type="hsv",
                        kernel_size=(5,5),
                        area_thresh=(400,50000)):
        if filter_type == "hsv":
            frame_hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
            mask = cv.inRange(frame_hsv,l_bound,u_bound)
            kernel = np.ones(kernel_size, np.uint8)
            frame_contoured = frame.copy()
            frame_rectangled = frame.copy()
            if dilate:
                mask_dilated = cv.dilate(mask, kernel, iterations=dilate_iteration)
                contours, _ = cv.findContours(mask_dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
            else:
                contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

            if draw_contours:
                cv.drawContours(frame_contoured, contours, -1, (255, 0, 0), 2)
            if draw_rectangle:
                x_list, y_list = Detector.drawRectangles(frame_rectangled,
                                                                            contours,
                                                                            area_l_thresh = area_thresh[0],
                                                                            area_u_thresh = area_thresh[1])

            return frame_rectangled, frame_contoured, frame_hsv, mask, x_list, y_list
        elif filter_type == "bgr":
            frame_bgr = frame.copy()
            mask = cv.inRange(frame_bgr,l_bound,u_bound)
            kernel = np.ones(kernel_size, np.uint8)
            frame_contoured = frame.copy()
            if dilate:
                mask_dilated = cv.dilate(mask, kernel, iterations=1)
                contours, _ = cv.findContours(mask_dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
            else:
                contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

            if draw_contours:
                cv.drawContours(frame_contoured, contours, -1, (255, 0, 0), 2)
            if draw_rectangle:
                x_list, y_list, frame_rectangled = Detector.drawRectangles(frame_contoured,
                                                                            contours,
                                                                            area_l_thresh = area_thresh[0],
                                                                            area_u_thresh = area_thresh[1])

            return frame_rectangled, frame_contoured, frame_bgr, mask, x_list, y_list
        
        elif filter_type == "gray":
            frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            mask = cv.inRange(frame_gray,l_bound,u_bound)
            kernel = np.ones(kernel_size, np.uint8)
            frame_contoured = frame.copy()
            if dilate:
                mask_dilated = cv.dilate(mask, kernel, iterations=1)
                contours, _ = cv.findContours(mask_dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
            else:
                contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

            if draw_contours:
                cv.drawContours(frame_contoured, contours, -1, (255, 0, 0), 2)
            if draw_rectangle:
                x_list, y_list, frame_rectangled = Detector.drawRectangles(frame_contoured,
                                                                            contours,
                                                                            area_l_thresh = area_thresh[0],
                                                                            area_u_thresh = area_thresh[1])
                                                                            
            return frame_rectangled, frame_contoured, frame_gray, mask, x_list, y_list
    @staticmethod
    def houghCircles(frame, mask, blur_size=5,param1=100,param2=15,minRadius=5,maxRadius=20,min_dist_denominator=16):
        frame_circled = frame.copy()
        res = cv.bitwise_and(frame,frame,mask=mask)
        frame_gray = res[:,:,2]
        frame_gray = cv.medianBlur(frame_gray, blur_size)
        circles = cv.HoughCircles(frame_gray,
                                    cv.HOUGH_GRADIENT, 1, int(frame.shape[0]/min_dist_denominator),
                                    param1=param1,
                                    param2=param2,
                                    minRadius=minRadius,
                                    maxRadius=maxRadius)
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                center = (i[0], i[1])
                # circle center
                cv.circle(frame_circled, center, 1, (0, 100, 100), 3)
                # draw line to the center
                frame_circled = Detector.drawLine(frame_circled,center)
                # circle outline
                radius = i[2]
                cv.circle(frame_circled, center, radius, (255, 0, 255), 3)
        return frame_circled

    def __init__(self,vid_name):
        self.vid_name = vid_name

    def preview(self):
        cap = cv.VideoCapture(self.vid_name)
        while cap.isOpened():
            ret, bgr = cap.read()
            hsv = cv.cvtColor(bgr, cv.COLOR_BGR2HSV)
            gray = cv.cvtColor(bgr, cv.COLOR_BGR2GRAY)
            cv.imshow("bgr",bgr)
            cv.imshow("hsv",hsv)
            cv.imshow("gray",gray)
            if cv.waitKey(24) == 27:
                break
        cv.destroyAllWindows()
        cap.release()
        
    def findBounds(self,filter_type="hsv"):
        cap = cv.VideoCapture(self.vid_name)
        cv.namedWindow("Filter")
        if filter_type == "hsv":
            cv.createTrackbar("L_H","Filter",0,180,self.nothing)
            cv.createTrackbar("U_H","Filter",180,180,self.nothing)

            cv.createTrackbar("L_S","Filter",0,255,self.nothing)
            cv.createTrackbar("U_S","Filter",255,255,self.nothing)

            cv.createTrackbar("L_V","Filter",0,255,self.nothing)
            cv.createTrackbar("U_V","Filter",255,255,self.nothing)

            while(True):
                ret, frame = cap.read()
                if ret:
                    frame_hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)

                    l_h = cv.getTrackbarPos("L_H","Filter")
                    u_h = cv.getTrackbarPos("U_H","Filter")

                    l_s = cv.getTrackbarPos("L_S","Filter")
                    u_s = cv.getTrackbarPos("U_S","Filter")

                    l_v = cv.getTrackbarPos("L_V","Filter")
                    u_v = cv.getTrackbarPos("U_V","Filter")

                    l_bound_hsv = np.array([l_h,l_s,l_v])
                    u_bound_hsv = np.array([u_h,u_s,u_v])

                    mask = cv.inRange(frame_hsv, l_bound_hsv, u_bound_hsv)

                    cv.imshow("frame_hsv",frame_hsv)
                    cv.imshow("mask",mask)
                    res = cv.bitwise_and(frame_hsv,frame_hsv,mask=mask)
                    cv.imshow("res",res)
                else:
                    cap.set(cv.CAP_PROP_POS_FRAMES, 0)
                key = cv.waitKey(24)
                if key==27:
                    break
            cap.release()
            cv.destroyAllWindows()
            return l_bound_hsv, u_bound_hsv


        elif filter_type == "bgr":
            cv.createTrackbar("L_B","Filter",0,255,self.nothing)
            cv.createTrackbar("U_B","Filter",255,255,self.nothing)

            cv.createTrackbar("L_G","Filter",0,255,self.nothing)
            cv.createTrackbar("U_G","Filter",255,255,self.nothing)

            cv.createTrackbar("L_R","Filter",0,255,self.nothing)
            cv.createTrackbar("U_R","Filter",255,255,self.nothing)

            while(True):
                ret, frame = cap.read()
                if ret:

                    l_b = cv.getTrackbarPos("L_B","Filter")
                    u_b = cv.getTrackbarPos("U_B","Filter")

                    l_g = cv.getTrackbarPos("L_G","Filter")
                    u_g = cv.getTrackbarPos("U_G","Filter")

                    l_r = cv.getTrackbarPos("L_R","Filter")
                    u_r = cv.getTrackbarPos("U_R","Filter")

                    l_bound_bgr = np.array([l_b,l_g,l_r])
                    u_bound_bgr = np.array([u_b,u_g,u_r])

                    mask = cv.inRange(frame,l_bound_bgr,u_bound_bgr)
                    cv.imshow("frame",frame)
                    cv.imshow("mask",mask)
                    res = cv.bitwise_and(frame,frame,mask=mask)
                    cv.imshow("res",res)
                else:
                    cap.set(cv.CAP_PROP_POS_FRAMES, 0)

                key = cv.waitKey(24)
                if key==27:
                    break
            cap.release()
            cv.destroyAllWindows()
            return l_bound_bgr, u_bound_bgr
            
        elif filter_type == "gray":
            cv.createTrackbar("L_G","Filter",0,255,self.nothing)
            cv.createTrackbar("U_G","Filter",255,255,self.nothing)

            while(True):
                ret, frame = cap.read()
                if ret:
                    frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

                    l_bound_gray = cv.getTrackbarPos("L_G","Filter")
                    u_bound_gray = cv.getTrackbarPos("U_G","Filter")

                    l_bound = np.array([l_bound_gray])
                    u_bound = np.array([u_bound_gray])

                    mask = cv.inRange(frame,l_bound,u_bound)
                    cv.imshow("frame",frame)
                    cv.imshow("mask",mask)
                    res = cv.bitwise_and(frame,frame,mask=mask)
                    cv.imshow("res",res)
                else:
                    cap.set(cv.CAP_PROP_POS_FRAMES, 0)
                key = cv.waitKey(24)
                if key==27:
                    break
            cap.release()
            cv.destroyAllWindows()
            return l_bound_gray, u_bound_gray
        else:
            print("Try proper filter_type")
            
    def findContoursLive(self,dilate=True,
                        l_bound=np.array([0,0,0]),
                        u_bound=np.array([0,0,0]),
                        draw_contours=False,
                        draw_rectangle=True,
                        filter_type="hsv",
                        line=True,
                        area_thresh=(400,50000),
                        dilate_iteration=1,
                        hough_circles=True):
        cap = cv.VideoCapture(self.vid_name)
        if filter_type == "hsv":
            while(True):
                ret, frame = cap.read()
                if ret:
                    frame_rectangled, frame_contoured, frame_hsv, mask, xs, ys = Detector.findContours( frame,
                                                                                                        l_bound=l_bound,
                                                                                                        u_bound=u_bound,
                                                                                                        filter_type="hsv",
                                                                                                        area_thresh=area_thresh,
                                                                                                        draw_contours=draw_contours,
                                                                                                        dilate_iteration=dilate_iteration,
                                                                                                        )
                    if line:
                        frame_rectangled_lined = self.drawLineRect(frame_rectangled,xs,ys)
                        cv.imshow("frame_rectangled_lined",frame_rectangled_lined)
                    else:
                        cv.imshow("frame_rectangled", frame_rectangled)
                    if hough_circles:
                        frame_circled = frame.copy()
                        res = cv.bitwise_and(frame,frame,mask=mask)
                        frame_gray = res[:,:,2]
                        frame_gray = cv.medianBlur(frame_gray, 5)
                        circles = cv.HoughCircles(frame_gray,
                                                    cv.HOUGH_GRADIENT, 1, 10,
                                                    param1=100,
                                                    param2=15,
                                                    minRadius=5,
                                                    maxRadius=20)
                        if circles is not None:
                            circles = np.uint16(np.around(circles))
                            for i in circles[0, :]:
                                center = (i[0], i[1])
                                # circle center
                                cv.circle(frame_circled, center, 1, (0, 100, 100), 3)
                                if line:
                                    # draw line to the center
                                    frame_circled = Detector.drawLine(frame_circled,center)
                                # circle outline
                                radius = i[2]
                                cv.circle(frame_circled, center, radius, (255, 0, 255), 3)
                        cv.imshow("frame_circled",frame_circled)
                    #cv.imshow("frame", frame)
                    #cv.imshow("mask",mask)
                    #cv.imshow("frame_hsv",frame_hsv)
                    #cv.imshow("frame_contoured",frame_contoured)
                    
                    
                else:
                    cap.set(cv.CAP_PROP_POS_FRAMES, 0)
                key = cv.waitKey(24)
                if key==27:
                    break
            cap.release()
            cv.destroyAllWindows()
        elif filter_type == "bgr":
            while(True):
                ret, frame = cap.read()
                if ret:
                    frame_rectangled, frame_contoured, frame_bgr, mask, xs, ys = Detector.findContours( frame,
                                                                                                        l_bound=l_bound,
                                                                                                        u_bound=u_bound,
                                                                                                        filter_type="bgr",
                                                                                                        area_thresh=area_thresh,
                                                                                                        draw_contours=draw_contours,
                                                                                                        dilate_iteration=dilate_iteration)
                    if line:
                        frame_rectangled_lined = self.drawLineRect(frame_rectangled,xs,ys)
                    #cv.imshow("frame", frame)
                    #cv.imshow("mask",mask)
                    #cv.imshow("frame_bgr",frame_bgr)
                    #cv.imshow("frame_contoured",frame_contoured)
                    #cv.imshow("frame_rectangled", frame_rectangled)
                    cv.imshow("frame_rectangled_lined",frame_rectangled_lined)
                else:
                    cap.set(cv.CAP_PROP_POS_FRAMES, 0)
                key = cv.waitKey(24)
                if key==27:
                    break
            cap.release()
            cv.destroyAllWindows()
            
        elif filter_type == "gray":
            while(True):
                ret, frame = cap.read()
                if ret:
                    frame_rectangled, frame_contoured, frame_gray, mask, xs, ys = Detector.findContours(frame,
                                                                                                        l_bound=l_bound,
                                                                                                        u_bound=u_bound,
                                                                                                        filter_type="gray", 
                                                                                                        area_thresh= area_thresh, 
                                                                                                        draw_contours=draw_contours,
                                                                                                        dilate_iteration=dilate_iteration)
                    if line:
                        frame_contoured_lined = self.drawLineRect(frame_contoured,xs,ys)
                    #cv.imshow("frame", frame)
                    #cv.imshow("mask",mask)
                    #cv.imshow("frame_gray",frame_gray)
                    #cv.imshow("frame_contoured",frame_contoured)
                    #cv.imshow("frame_rectangled", frame_rectangled)
                    cv.imshow("frame_rectangled_lined",frame_rectangled_lined)
                else:
                    cap.set(cv.CAP_PROP_POS_FRAMES, 0)
                    
                key = cv.waitKey(24)
                if key==27:
                    break
            cap.release()
            cv.destroyAllWindows()
            
          
   

In [40]:
x = Detector("para.mp4")

In [16]:
x.preview()

In [41]:
l_bound_hsv ,u_bound_hsv = x.findBounds(filter_type="hsv")

In [42]:
x.findContoursLive(l_bound=l_bound_hsv,u_bound=u_bound_hsv,draw_contours=False, filter_type = "hsv",dilate_iteration=1,line=True)

In [34]:
int(frame.shape[0]/16)

40

In [8]:
cap = cv.VideoCapture("bluedetection.mp4")
while(True):
    ret, frame = cap.read()
    if ret:
        frame_circled = frame.copy()
        frame_rectangled, frame_contoured, _, mask_hsv, x_list, y_list = Detector.findContours(frame,
                                                                                    l_bound=l_bound_hsv,
                                                                                    u_bound=u_bound_hsv,
                                                                                    draw_contours=False,
                                                                                    area_thresh=(650,900))
        res = cv.bitwise_and(frame,frame,mask=mask_hsv)
        frame_gray = res[:,:,2]
        frame_gray = cv.medianBlur(frame_gray, 5)
        circles = cv.HoughCircles(frame_gray,
                                    cv.HOUGH_GRADIENT, 1, 10,
                                    param1=100,
                                    param2=15,
                                    minRadius=5,
                                    maxRadius=20)
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                center = (i[0], i[1])
                # circle center
                cv.circle(frame_circled, center, 1, (0, 100, 100), 3)
                # draw line to the center
                frame_circled = Detector.drawLine(frame_circled,center)
                # circle outline
                radius = i[2]
                cv.circle(frame_circled, center, radius, (255, 0, 255), 3)
        canny = cv.Canny( frame_gray, 40, 80 )
        #cv.imshow("canny",canny)
        #cv.imshow("frame_gray",frame_gray)
        #cv.imshow("frame", frame)
        cv.imshow("frame_rectangled",frame_rectangled)
        #cv.imshow("frame_contoured",frame_contoured)
        #cv.imshow("mask_hsv", mask_hsv)
        #cv.imshow("res",res)
        cv.imshow("frame_circled",frame_circled)
    else:
        cap.set(cv.CAP_PROP_POS_FRAMES, 0)
    key = cv.waitKey(4)
    if key==27:
        break
cap.release()
cv.destroyAllWindows()